我们来实现如果文章没有输入摘要，则自动摘取正文的前 N 个字符作为摘要，这有两种实现方法。

# 复写 save 方法

通过复写模型的 save 方法，从正文字段摘取前 N 个字符保存到摘要字段。

In [ ]:
# blog/models.py

import markdown
from django.utils.html import strip_tags

class Post(models.Model):
    # 其它字段...
    body = models.TextField()
    excerpt = models.CharField(max_length=200, blank=True)

    # 其它方法...

    def save(self, *args, **kwargs):    
        # 如果没有填写摘要
        if not self.excerpt:
            # 首先实例化一个 Markdown 类，用于渲染 body 的文本
            md = markdown.Markdown(extensions=[
                'markdown.extensions.extra',
                'markdown.extensions.codehilite',
            ])
            # 先将 Markdown 文本渲染成 HTML 文本
            # strip_tags 去掉 HTML 文本的全部 HTML 标签
            # 从文本摘取前 54 个字符赋给 excerpt
            self.excerpt = strip_tags(md.convert(self.body))[:54]

        # 调用父类的 save 方法将数据保存到数据库中
        super(Post, self).save(*args, **kwargs)

先将 body 中的 Markdown 文本转为 HTML 文本，去掉 HTML 文本里的 HTML 标签，然后摘取文本的前 54 个字符作为摘要。去掉 HTML 标签的目的是防止前 54 个字符中存在块级 HTML 标签而使得摘要格式比较难看。

# 使用 truncatechars 模板过滤器

使用 truncatechars 模板过滤器（Filter）

在 Django 的模板系统中，模板过滤器的使用语法为 {{ var | filter: arg }}。可以将模板过滤看做一个函数，它会作用于被它过滤的模板变量，从而改变模板变量的值。

```HTML
templates/blog/index.html

<article class="post post-{{ post.pk }}">
  ...
  <div class="entry-content clearfix">
      <p>{{ post.body|truncatechars:54 }}</p>
      <div class="read-more cl-effect-14">
          <a href="{{ post.get_absolute_url }}" class="more-link">继续阅读 <span class="meta-nav">→</span></a>
      </div>
  </div>
</article>
```

不过这种方法的一个缺点就是如果前 54 个字符含有块级 HTML 元素标签的话（比如一段代码块），会使摘要比较难看。所以推荐使用第一种方法。